In [ ]:
spark.sparkContext.addPyFile("https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.1.2/spark-avro_2.12-3.1.2.jar")

In [ ]:
def read_file(spark, header, encoding, infer_schema, bucket_name, file_path):
    return spark.read \
        .option('header', header) \
        .option('encoding', encoding) \
        .option('inferSchema', infer_schema) \
        .csv(f's3://{bucket_name}/{file_path}') \
        .cache()

In [ ]:
def write_table(spark, df, bucket_name, database, table_name, file_format, save_as_table):
    path = f's3://{bucket_name}/benchmark/{database}/{table_name}_{file_format}'
    if save_as_table:
        return df.write.mode('overwrite').format(file_format).option('path', path).saveAsTable(f'{database}.{table_name}_{file_format}')
    return df.write.mode('overwrite').format(file_format).option('path', path).save()

In [ ]:
bucket_name = 'eedb-006-2024-2-grupo-2'
database = 'eedb_006'
table_name = 'inep'
file_path = 'incoming/NIVELSUPERIORNORMALIZADA2.csv'

df = read_file(spark=spark, header=True, encoding='UTF-8', infer_schema=True, bucket_name=bucket_name, file_path=file_path)

df.show(truncate=False, vertical=True, n=1)

In [ ]:
df.dtypes

In [ ]:
write_table(spark, df, bucket_name, database, table_name, 'parquet', True)
write_table(spark, df, bucket_name, database, table_name, 'csv', False)
write_table(spark, df, bucket_name, database, table_name, 'json', False)
write_table(spark, df, bucket_name, database, table_name, 'avro', True)